In [94]:
!kaggle datasets download msambare/fer2013

fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import os
import zipfile

# Specify the path to the zip file
zip_file_path = 'fer2013.zip'

# Specify the path to the destination folder
destination_folder = 'dataset'

# Check if the destination folder exists, create it if it doesn't
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)
    

In [2]:
import torchvision.datasets as dataset
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.RandomResizedCrop(128, ratio=(0.95, 1.05)),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

# Create the dataset 
dataset_train = dataset.ImageFolder(root='dataset/train', transform=transform)
dataset_test = dataset.ImageFolder(root='dataset/test', transform=transform)

/Users/paolomazzitti/anaconda3/envs/laurea/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/paolomazzitti/anaconda3/envs/laurea/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/paolomazzitti/anaconda3/envs/laurea/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <AD0702F8-F0F4-3872-8C19-A834018634B4> /Users/paolomazzitti/anaconda3/envs/laurea/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
train_dir = 'dataset/train'
num_images_per_class = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in dataset_train.classes}
print(num_images_per_class)

{'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}


#### Oversampling

In [4]:
import torch
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader

class_counts = torch.tensor([num_images_per_class[cls] for cls in dataset_train.classes], dtype=torch.float)
class_weights = 1. / class_counts
weights = torch.tensor([class_weights[cls] for _, cls in dataset_train.samples])
sampler = WeightedRandomSampler(weights, len(weights)) # type: ignore
train_loader = DataLoader(dataset_train, batch_size=16, sampler=sampler)
test_loader = DataLoader(dataset_test, batch_size=1)

In [5]:
import torch
from itertools import islice

var_counts = []
for i, (X_batch, y_batch) in enumerate(islice(train_loader, 5)):
    bin_count = torch.bincount(y_batch).type(torch.float)
    var_counts.append(torch.std(bin_count))
    print(bin_count)
var_counts_tensor = torch.tensor(var_counts)
print(torch.mean(var_counts_tensor))

tensor([1., 0., 3., 3., 5., 1., 3.])
tensor([3., 1., 3., 3., 3., 1., 2.])
tensor([2., 0., 3., 2., 0., 6., 3.])
tensor([2., 3., 4., 0., 2., 4., 1.])
tensor([2., 0., 5., 1., 3., 2., 3.])
tensor(1.5628)


In [6]:
len(dataset_train)

28709

In [26]:
from torch import nn
from torch.nn import init

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        init.normal_(m.weight, mean=0, std=0.05)
        init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        init.normal_(m.weight, mean=0, std=0.05)
        init.constant_(m.bias, 0)

class ResBlock(nn.Module):
    def __init__(self, input_channels, output_channels, dropout):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(input_channels, input_channels, kernel_size=2, stride=2, padding='valid'),
            nn.BatchNorm2d(input_channels),
            nn.ELU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, input_channels, kernel_size=3, padding='same', dilation=2),
            nn.BatchNorm2d(input_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, input_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(input_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, output_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(output_channels),
            nn.Dropout(dropout),
        )

        self.identity = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=2, stride=2, padding='valid'),
            nn.BatchNorm2d(output_channels),
        )

        self.apply(init_weights)

    def forward(self, x):
        x1 = self.sequential(x)
        x2 = self.identity(x)
        res = x1 + x2
        res = torch.nn.functional.relu(res)
        return res

In [27]:
from torch import nn

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        init.normal_(m.weight, mean=0, std=0.05)
        init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        init.normal_(m.weight, mean=0, std=0.05)
        init.constant_(m.bias, 0)

class IdentityBlock(nn.Module):
    def __init__(self, input_channels, output_channels, dropout):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(output_channels, input_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(input_channels),
            nn.ELU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, input_channels, kernel_size=3, padding='same', dilation=2),
            nn.BatchNorm2d(input_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, input_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(input_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv2d(input_channels, output_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(output_channels),
            nn.Dropout(dropout),
        )

        self.apply(init_weights)

    def forward(self, x):
        x1 = self.sequential(x)
        res = x1 + x
        res = torch.nn.functional.relu(res)
        return res


In [28]:
class Lemon(nn.Module):
    def __init__(self):
        super().__init__()
        self.stage_1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding='same'),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )

        self.stage_2 = nn.Sequential(
            ResBlock(32, 48, 0.3),
            IdentityBlock(32, 48, 0.2)
        )

        self.stage_3 = nn.Sequential(
            ResBlock(48, 64, 0.2),
            IdentityBlock(48, 64, 0.2)
        )

        self.stage_4 = nn.Sequential(
            ResBlock(64, 80, 0.1),
            IdentityBlock(64, 80, 0.1)
        )

        self.stage_5 = nn.Sequential(
            ResBlock(80, 96, 0.1),
            IdentityBlock(80, 96, 0.1)
        )

        self.stage_6 = nn.Sequential(
            ResBlock(96, 112, 0.0),
            IdentityBlock(96, 112, 0.0)
        )

        self.stage_7 = nn.Sequential(
            nn.AvgPool2d(2),
            nn.Flatten(),
            nn.Linear(112, 144),
            nn.Linear(144, 7)
        )

    def forward(self, x):
        x = self.stage_1(x)
        x = self.stage_2(x)
        x = self.stage_3(x)
        x = self.stage_4(x)
        x = self.stage_5(x)
        x = self.stage_6(x)
        x = self.stage_7(x)
        return x

In [14]:
def get_net():
    net = nn.Sequential(
        nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding='same'),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        ),
        nn.Sequential(
            nn.AvgPool2d(2),
            nn.Flatten(),
            nn.Linear(112, 144),
            nn.Linear(144, 7)
        ))
    return net

model = get_net()
model = torch.jit.script(model)

In [30]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
model = Lemon()
model = torch.jit.script(model)

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.001)
loss_fn = CrossEntropyLoss()
state_dict = {}

train_losses = []
test_losses = []

# Addestramento
for epoch in range(100):
    state_dict['epoch'] = epoch
    train_loss = torch.Tensor()
    model.train()
    total_train_loss = 0
    total_samples = 0
    print(f" -- Epoch {epoch} --")
    for X_batch, y_batch in tqdm(train_loader):

        optimizer.zero_grad()
        predictions = model(X_batch)
        train_loss = loss_fn(predictions, y_batch)
        train_loss.backward()
        total_train_loss += train_loss.item() * X_batch.size(0)
        total_samples += X_batch.size(0)
        optimizer.step()
    average_train_loss = total_train_loss / total_samples
    train_losses.append(average_train_loss)
    print(f"Train loss: {average_train_loss}")

    model.eval()
    
    total_samples = 0
    loss_fn = CrossEntropyLoss()
    test_losses = []
    total_test_loss = 0

    with torch.inference_mode():
        for X_batch, y_batch in test_loader:
            predictions = model(X_batch)
            loss = loss_fn(predictions, y_batch)
            total_test_loss += loss.item() * X_batch.size(0)
            total_samples += X_batch.size(0)
        average_test_loss = total_test_loss / total_samples
        test_losses.append(average_test_loss)
        print(f"Test loss: {average_test_loss}")

    state_dict['model_state_dict'] = model.state_dict()
    state_dict['optimizer_state_dict'] = optimizer.state_dict()
    state_dict['train_losses'] = train_losses
    state_dict['test_losses'] = test_losses
    torch.save(state_dict, "lemon.pth")

 -- Epoch 0 --


  0%|          | 0/1795 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
from torchinfo import summary

summary(Lemon(), input_size=(32, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
Lemon                                    [32, 7]                   --
├─Sequential: 1-1                        [32, 32, 64, 64]          --
│    └─Conv2d: 2-1                       [32, 32, 128, 128]        896
│    └─BatchNorm2d: 2-2                  [32, 32, 128, 128]        64
│    └─ELU: 2-3                          [32, 32, 128, 128]        --
│    └─MaxPool2d: 2-4                    [32, 32, 64, 64]          --
│    └─Dropout: 2-5                      [32, 32, 64, 64]          --
├─Sequential: 1-2                        [32, 48, 32, 32]          --
│    └─ResBlock: 2-6                     [32, 48, 32, 32]          --
│    │    └─Sequential: 3-1              [32, 48, 32, 32]          36,784
│    │    └─Sequential: 3-2              [32, 48, 32, 32]          6,288
│    └─IdentityBlock: 2-7                [32, 48, 32, 32]          --
│    │    └─Sequential: 3-3              [32, 48, 32, 32]          46,512
├─S

In [1]:
import flax
import jax

ModuleNotFoundError: No module named 'flax'